In [1]:
import snap
import pandas as pd
import numpy as np
import csv
import collections
import operator

In [2]:
df_citation = pd.read_table('~/uspatentcitation.tsv', usecols=['patent_id', 'citation_id'])

In [3]:
df_citation.head(4)

,patent_id,citation_id
0,9009250,8127342
1,9643605,5471515
2,5354551,4875247
3,D786922,D718330


In [4]:
#patent_id cited #citation_id which was released on #date
#check patent_id is released between 2000 and 2010
#check citation_id is released between 1990 and 2000. Find out which #citation_id belongs to which company
#########################################################################################################
patentid_col = df_citation['patent_id']
citationid_col = df_citation['citation_id']

In [5]:
#print sorted(patentid_col)

In [6]:
citation_cache = collections.defaultdict(list) #map cited patent to all patents that cited it
for i in range(len(citationid_col)):
    patent_cited = citationid_col[i]
    citation_cache[patent_cited].append(patentid_col[i])
#print citation_cache

In [7]:
df_filtered = pd.read_table('~/patent_organization_filtered.tsv', usecols=['patent_id', 'organization'])
df_filtered.head(4)

,patent_id,organization
0,D356598,The Gillette Company
1,D356598,The Gillette Company
2,D356598,The Gillette Company
3,D356598,The Gillette Company


In [8]:
org_patent_cache = collections.defaultdict(list) #map each firm to the patents that it owns. patent already filtered to be in 1990-2000
patentid_col = df_filtered['patent_id']
organization_col = df_filtered['organization']

for i in range(len(patentid_col)):
    org = organization_col[i]
    cited_patent = patentid_col[i]
    org_patent_cache[org].append(cited_patent)

In [9]:
df_patent = pd.read_table('~/patent.tsv', usecols=['id', 'date'])
df_patent.head(4)

/Users/Thao/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,date
0,3930271,1976-01-06
1,3930272,1976-01-06
2,3930273,1976-01-06
3,3930274,1976-01-06


In [15]:
id_col = df_patent['id']
date_col = df_patent['date']
patents_00_10 = [] #patents between year 2000-2010

for i in range(len(id_col)):
    patent = id_col[i]
    date = date_col[i]
    if not isinstance(date, str):
        continue
    year = date.split('-')[0]
    if not year.isdigit():
        print year
        continue
    if year >= 2000 and year <= 2010:
        patents_00_10.append(patent)

pg030624.zip


In [20]:
citedpatent_count = collections.defaultdict(int)
for cited_patent in citation_cache:
    cite_list = citation_cache[cited_patent]
    for patent in cite_list:
        if patent not in patents_00_10:
            cite_list.remove(patent)
    citedpatent_count[cited_patent] += len(cite_list)
    

In [21]:
#print sorted(patentid_col) #seems weird - only starts with letter

In [22]:
org_ncitations = collections.defaultdict(dict)
for org, patent_list in org_patent_cache.items():
    for patent in patent_list:
        org_ncitations[org][patent] = citedpatent_count[patent]

In [23]:
#find total count of forward citations
#compute h_index
f = open('CitationCount_HIndex.csv', 'a')
writer = csv.writer(f)

for org, citation_count_dict in org_ncitations.items():
    count_values = citation_count_dict.values()
    sorted_count_values = sorted(count_values, reverse=True) #descending
    #sorted_citation_count = sorted(citation_count_dict, key=operator.itemgetter(0), reverse=True) #descending order
    total_count = sum(sorted_count_values)
    h_index = len(sorted_count_values)
    for i in range(len(sorted_count_values)):
        if sorted_count_values[i] >= i+1:
            continue
        else:
            h_index = i
            break
    writer.writerow([org, total_count, h_index])

f.close()